# Machine Learning Modeling — SurviveAI: Predicting Human Survival
Ya tenemos los datos explorados y limpios. Ahora vamos a resumir los hallazgos clave y proponer próximos pasos.


## Cargar datos y separar X y y

Aqui le decimos a la maquina que mirar (X) y que predecir (y).

In [14]:
import pandas as pd

df= pd.read_csv('../Data/train_clean.csv')

target = 'survived'

y= df[target].astype('int')
X= df.drop(columns=[target])    




### Explicacion facil de entender

- X: Estas son las características o atributos que usaremos para hacer predicciones. En este caso, incluyen información sobre los pasajeros del Titanic, como su edad, sexo, clase de pasajero, tarifa pagada, etc. Estas características nos ayudan a entender el contexto y las condiciones de cada pasajero.
- y: Esta es la variable objetivo que queremos predecir. En este caso, es si un pasajero sobrevivió o no al desastre del Titanic. Es una variable binaria donde 1 significa que el pasajero sobrevivió y 0 significa que no sobrevivió. Nuestro objetivo es entrenar un modelo de machine learning para predecir esta variable basándonos en las características (X) de los pasajeros.

Eliminamos la columna target de X porque es la variable que queremos predecir, y no debe ser parte de las características que usamos para hacer la predicción. 


## DIVIDIR EN TRAIN Y TEST
El modelo debe aprender de datos que no ha visto antes. Por eso dividimos los datos en dos partes: una para entrenar el modelo (train) y otra para probar su rendimiento (test). Usamos el 80% de los datos para entrenar y el 20% para probar.

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42 , stratify=y)



### Explicacion facil de entender

Basicamente, al dividir los datos en conjuntos de entrenamiento y prueba, nos aseguramos de que el modelo pueda generalizar bien a datos nuevos y no solo memorizar los datos con los que fue entrenado. Esto es crucial para evaluar su rendimiento de manera precisa.

train el gimnasio donde el modelo aprende, y test es como un examen para ver que tanto aprendio.

test_size=0.2 significa que el 20% de los datos se usaran para probar el modelo, y el 80% restante para entrenarlo.

stratify=y asegura que la proporción de sobrevivientes y no sobrevivientes sea la misma en ambos conjuntos, lo que ayuda a mantener el equilibrio de clases durante el entrenamiento y la prueba.


## FEATURE ENGINEERING 

### Explicacion facil de entender
El feature engineering es el proceso de transformar y crear nuevas características a partir de los datos originales para mejorar el rendimiento de un modelo de machine learning. En este caso, hemos realizado varias transformaciones en las características de los pasajeros del Titanic para hacerlas más útiles para el modelo. Estas transformaciones incluyen:
- Crear una nueva característica que representa el tamaño de la familia del pasajero.
- Indicar si el pasajero viajaba solo o acompañado.
- Calcular la tarifa promedio por persona en la cabina.
- Extraer el título del nombre del pasajero (por ejemplo, "Mr", "Mrs", "Miss") para capturar información sobre su estatus social.
Estas nuevas características pueden proporcionar información adicional que ayuda al modelo a aprender patrones más complejos y mejorar su capacidad para predecir si un pasajero sobrevivió o no al desastre del Titanic.

In [16]:
import numpy as np
import re 

def add_features(df):
    
    # Creamos nuevas características basadas en las existentes
    
    # COPIAR EL DATAFRAME PARA EVITAR MODIFICAR EL ORIGINAL
    df_clean = df.copy()
    
    
    # Tamano de la familia
    
    df_clean["family_size"] = df_clean["sibsp"] + df_clean["parch"] + 1 # +1 para incluir al propio pasajero
    
    # Viaja solo o acompañado

    df_clean["is_alone"] =  (df_clean["family_size"] == 1).astype(int) #ponemos astype int para convertir True/False a 1/0

    
    # Tarifa por persona

    df_clean ["fare_per_person"] = df_clean["fare"] / df_clean["family_size"] # dividimos la tarifa entre el tamano de la familia
    
    
    # Extraer el titulo del nombre
    
    df_clean['title'] = df_clean['name'].str.extract(r',\s*([^\.]+)\.', expand=False)
    df_clean['title'] = df_clean['title'].replace(['Mlle','Ms'],'Miss').replace('Mme','Mrs')
    return df_clean

X_train = add_features(X_train)
X_test = add_features(X_test)



# 1) Quitar identificadores crudos (no informativos, causan overfitting)
leak_cols = ["name", "ticket", "passengerid"]
for c in leak_cols:
    if c in X_train.columns:
        X_train = X_train.drop(columns=c)
    if c in X_test.columns:
        X_test = X_test.drop(columns=c)

# 2) Renombrar columnas para consistencia
if "Has_Cabin" in X_train.columns and "has_cabin" not in X_train.columns:
    X_train = X_train.rename(columns={"Has_Cabin": "has_cabin"})
    X_test  = X_test.rename(columns={"Has_Cabin": "has_cabin"})


## SEPARAR COLUMNAS NUMERICAS Y CATEGORICAS
Para facilitar el procesamiento de los datos, separamos las columnas en dos grupos: numéricas y categóricas. Las columnas numéricas contienen valores cuantitativos (como edad, tarifa, etc.), mientras que las columnas categóricas contienen valores cualitativos (como sexo, clase de pasajero, etc.). Esta separación nos permite aplicar diferentes técnicas de preprocesamiento a cada tipo de dato.



In [17]:
num_cols = [c for c in X_train.columns if X_train[c].dtype.kind in "if"]
cat_cols = [c for c in X_train.columns if c not in num_cols]


La primera linea crea una lista de nombres de columnas que contienen datos numéricos (tipos de datos enteros y flotantes). La segunda línea crea una lista de nombres de columnas que no están en la lista de columnas numéricas, es decir, las columnas categóricas.



## PIPELINE
Un pipeline es una secuencia de pasos que se aplican a los datos para prepararlos y entrenar un modelo de machine learning. 

Es importante porque nos permite organizar y automatizar el proceso de preprocesamiento y modelado de datos. Al usar un pipeline, podemos asegurarnos de que los mismos pasos se apliquen de manera consistente tanto durante el entrenamiento del modelo como durante la predicción en nuevos datos. Esto ayuda a evitar errores y facilita la reproducibilidad del modelo.

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

### Pipeline para columnas numericas 


Aqui definimos un pipeline para las columnas numéricas que incluye dos pasos:
1. Imputación: Rellena los valores faltantes con la mediana de la columna.
2. Escalado: Escala todos los numeros para que esten en un rango similar, lo que ayuda a mejorar el rendimiento del modelo.

In [19]:
num_tf =Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])


### Pipeline para columnas categoricas

In [20]:
cat_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])


Basically, este pipeline para las columnas categóricas también tiene dos pasos:
1. Imputación: Rellena los valores faltantes con la categoría más frecuente en cada columna.
2. One-Hot Encoding: Convierte las categorías en una representación binaria (0s y 1s) para que el modelo pueda entenderlas mejor.

### Combinar todo en un ColumnTransformer
Finalmente, combinamos ambos pipelines (numérico y categórico) en un ColumnTransformer. Esto nos permite aplicar automáticamente el preprocesamiento adecuado a cada tipo de columna cuando entrenamos el modelo o hacemos predicciones.

In [21]:
preprocess = ColumnTransformer([
    ('num', num_tf, num_cols),
    ('cat', cat_tf, cat_cols)
])

## PROBAR MODELOS
Aqui probamos varios modelos de machine learning para ver cual funciona mejor con nuestros datos. Usamos validación cruzada para evaluar el rendimiento de cada modelo de manera más robusta.

Primero , uno simple para tener una base , luego modelos mas complejos. Esto es para ver si los modelos mas complejos realmente aportan una mejora significativa en el rendimiento.

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

log_reg = Pipeline([
    ('prep', preprocess),
    ('clf', LogisticRegression(random_state=500))
])

rf = Pipeline([("prep", preprocess),
               ("clf", RandomForestClassifier(n_estimators=300, random_state=42))])

## VALIDACION CRUZADA
La validación cruzada es una técnica utilizada para evaluar el rendimiento de un modelo de machine learning de manera más robusta. En lugar de dividir los datos en un solo conjunto de entrenamiento y prueba, la validación cruzada divide los datos en varios subconjuntos (o "folds"). El modelo se entrena y evalúa múltiples veces, utilizando diferentes combinaciones de estos subconjuntos. Esto ayuda a asegurar que el modelo generalice bien a datos nuevos y no esté sobreajustado a un conjunto específico de datos.

In [23]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for name, model in {"Logistic": log_reg, "Random Forest": rf}.items():
    
    scc = cross_val_score(model, X_train, y_train, cv=skf, scoring='accuracy').mean()

    print(f"{name} Accuracy promedio: {scc:.3f}")

Logistic Accuracy promedio: 0.827
Random Forest Accuracy promedio: 0.816


### Resultados de la validacion cruzada

#### Regresion Logistica
Logistic Accuracy promedio: 0.829

Esto significa que el modelo de regresión logística tiene una precisión promedio del 82.9% en la predicción de si un pasajero sobrevivió o no al desastre del Titanic. En otras palabras, el modelo es capaz de clasificar correctamente el estado de supervivencia del 82.9% de los pasajeros en los datos de prueba durante la validación cruzada.

#### Random Forest
Random Forest Accuracy promedio: 0.817

Esto significa que el modelo de Random Forest tiene una precisión promedio del 81.7% en la predicción de si un pasajero sobrevivió o no al desastre del Titanic. En otras palabras, el modelo es capaz de clasificar correctamente el estado de supervivencia del 81.7% de los pasajeros en los datos de prueba durante la validación cruzada.

#### Comparacion
En este caso, el modelo de regresión logística tiene una precisión ligeramente superior (82.9%) en comparación con el modelo de Random Forest (81.7%). Esto sugiere que, al menos en este conjunto de datos y con la configuración actual, la regresión logística es un poco más efectiva para predecir la supervivencia de los pasajeros del Titanic que el modelo de Random Forest

Las relaciones entre variables como sexo , clase y edad son bastante lineales y directas, lo que favorece a la regresión logística.


## EVALUACION FINAL EN TEST 
Ahora si evaluamos el mejor modelo (Regresion Logistica) en el conjunto de test que no habia visto antes.

### Eleccion del modelo
Elegimos la regresion logistica porque tuvo mejor rendimiento en la validacion cruzada.

In [24]:
log_reg.fit(X_train, y_train)
from sklearn.metrics import accuracy_score, classification_report

y_pred = log_reg.predict(X_test)
print("Accuracy en test:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy en test: 0.8379888268156425
              precision    recall  f1-score   support

           0       0.86      0.88      0.87       110
           1       0.80      0.77      0.79        69

    accuracy                           0.84       179
   macro avg       0.83      0.82      0.83       179
weighted avg       0.84      0.84      0.84       179



### Interpretacion de resultados
Vemos que el logistic regression tuvo un accuracy de 0.84 en el conjunto de test, lo que indica que el modelo es capaz de clasificar correctamente el estado de supervivencia del 84% de los pasajeros en los datos de prueba.

#### Precision, Recall y F1-Score CLASE 0 
- Precision (Precisión): De todos los que el modelo dijo que murio el 87 % realmente murio. Esto significa que el modelo es bastante confiable cuando predice que un pasajero no sobrevivió.
- Recall (Sensibilidad): De todos los que realmente murieron, el modelo identifico correctamente , un 88 % de ellos. Esto indica que el modelo es efectivo para detectar a los pasajeros que no sobrevivieron.
- F1-Score: El F1-Score es una medida que combina la precisión y el recall en un solo valor. Un F1-Score alto (0.88 para la clase negativa) indica que el modelo tiene un buen equilibrio entre precisión y recall.
- Soporte: El soporte indica la cantidad de muestras reales en cada clase. En este caso, hay 110 pasajeros que no sobrevivieron y 58 que sí sobrevivieron en el conjunto de prueba.

#### Precision, Recall y F1-Score CLASE 1
- Precision (Precisión): De todos los que el modelo dijo que sobrevivio el 81 % realmente sobrevivio. Esto significa que el modelo es bastante confiable cuando predice que un pasajero sobrevivió.
- Recall (Sensibilidad): De todos los que realmente sobrevivieron, el modelo identifico correctamente , un 80 % de ellos. Esto indica que el modelo es efectivo para detectar a los pasajeros que sobrevivieron.
- F1-Score: El F1-Score es una medida que combina la precisión y el recall en un solo valor. Un F1-Score alto (0.80 para la clase positiva) indica que el modelo tiene un buen equilibrio entre precisión y recall.
- Soporte: El soporte indica la cantidad de muestras reales en cada clase. En este caso, hay 69 pasajeros que sobrevivieron.

#### Promedio generales 
- Accuray: Acierta en 85 de cada 100 predicciones.
- Macro avg: Promedio simple de precision, recall y f1-score sin considerar el soporte. Un 84 es bastante bueno.
- Weighted avg: Promedio ponderado de precision, recall y f1-score considerando el soporte de cada clase. Un 85 es bastante bueno. 


## SABER QUE VIO EL MODELO PARA DECIDIR


In [25]:
# Obtener nombres finales
ohe = log_reg.named_steps["prep"].named_transformers_["cat"].named_steps["ohe"]
cat_names = list(ohe.get_feature_names_out(cat_cols))
final_names = num_cols + cat_names

# Obtener coeficientes del modelo logístico
coefs = log_reg.named_steps["clf"].coef_[0]

# Crear DataFrame ordenado
pd.DataFrame({"Feature": final_names, "Coefficient": coefs}) \
  .sort_values("Coefficient", ascending=False) \
  .head(10)


,Feature,Coefficient
22,title_Master,1.342951
11,sex_female,0.828403
25,title_Mrs,0.685654
5,has_cabin,0.387301
21,title_Major,0.313811
14,embarked_Q,0.291117
27,title_Sir,0.259750
4,fare,0.146176
20,title_Lady,0.079688
16,title_Col,0.076728


 ### Importancia de las caracteristicas

El modelo de Regresión Logística muestra las variables que más influyen en la probabilidad de supervivencia. A continuación se interpretan los coeficientes positivos (que aumentan las probabilidades de sobrevivir) y los más relevantes del modelo final:

- title_Master (+1.34) → Los pasajeros con el título Master (niños varones) tenían alta probabilidad de sobrevivir, debido a la regla histórica “women and children first”.
- sex_female (+0.82) → Ser mujer incrementa fuertemente la probabilidad de supervivencia. Fue el factor más dominante en la mayoría de los casos.
- title_Mrs (+0.68) → Mujeres casadas (mayores o acompañadas) también mostraron alta tasa de supervivencia, en parte por estatus y prioridad en evacuación.
- has_cabin (+0.38) → Tener cabina indica mejores condiciones de alojamiento y proximidad a las cubiertas de rescate.
- title_Major (+0.31) → Aunque minoritario, este título refleja un estatus social alto, lo que podía facilitar el acceso a botes.
- embarked_Q (+0.29) → Pasajeros que embarcaron en Queenstown (Q) mostraron una leve mayor probabilidad, probablemente por diferencias en la composición social del grupo.
- title_Sir, title_Lady, title_Col → Títulos de nobleza o militares que reflejan posición privilegiada o acompañamiento, asociados con mejores resultados.
- fare (+0.14) → Un precio de boleto mayor tiende a correlacionarse con clases superiores, por lo tanto con mayor supervivencia.

## GUARDAR MODELO


In [26]:
from pathlib import Path
import joblib

model_path = Path("../models/surviveai_model.joblib")
model_path.parent.mkdir(parents=True, exist_ok=True)

joblib.dump(log_reg, model_path)
print(f"✅ Modelo guardado en: {model_path.resolve()}")


✅ Modelo guardado en: /Users/joseph/Documents/ia-projects/Kaggle/SurviveAI---Full-End-to-End-Explainable-Machine-Learning-System/models/surviveai_model.joblib
